# Retrieval Augmented Question & Answering with Amazon Bedrock using LangChain - Semantic search with metadata filtering with self-querying with ChromaDB - with Amazon Titan LLM

### Context
LLM question answering (Q+A) typically involves retrieval of documents relevant to the question followed by synthesis of the retrieved chunks into an answer by an LLM.  In practice, the retrieval step is necessary because the LLM context window is limited relative to the size of most text corpus of interest (e.g., LLM context windows range is limited to certain tokens). Anthropic recently released a Claude model with a 100k token context window.  With the advent of models with larger context windows, it is reasonable to wonder whether the document retrieval stage is necessary for many Q+A or chat use-cases.

One such retriever architectures with this retriever-less option is Semantic search with metadata filtering with self-querying option using Chroma database as vector database. 

### Pattern

Semantic with metadata filtering refers to the process of finding items that are both semantically relevant and meet specific filter conditions, such as the availability of a product in stock. In addition to the vector representation, the raw media format (e.g., document or image) contains a list of metadata, such as author and date.

To illustrate this, let's consider an example of image retrieval, specifically finding shoe images released after a certain date. Embedding the date directly into the vector representation is challenging, but we can overcome this obstacle by employing a combination of vector search and metadata filtering.

In this approach, we utilize the vector representation to perform similarity searches, looking for images that are visually similar to the query (shoe images). However, to fulfill the additional requirement of being released after a specific date, we leverage the metadata filtering step. By applying the date filter to the list of image metadata, we can identify and retrieve only the shoe images that match the desired release date criteria. This way, we effectively solve the problem of finding shoe images released after a particular date using a combination of vector search and metadata filtering.

Let's assume we have a music library with each song represented by a feature vector (encoded by a deep learning model) and accompanied by metadata such as "artist," "genre," and "release date." We'll explore three strategies for combining vector-based search and metadata filtering in this music library:

* Pre-selection filtering: In this approach, we first apply metadata filtering to identify a subset of songs that meet certain criteria, such as a specific artist or genre. We then mark these songs as eligible for vector-based similarity search. During the vector search process, we only consider the eligible songs and return the most similar songs from that subset.

* Post-selection filtering: Here, we begin by conducting a vector-based similarity search on the entire music library, finding a subset of songs that are semantically relevant to the query song. Once we have this subset, we then apply metadata filtering to further refine the results and obtain the final list of songs that match both the similarity and metadata criteria.

* Simultaneous search with filtering: In this strategy, we combine the vector search and metadata filtering steps during the search process itself. As we perform the similarity search on the feature vectors, we immediately filter out any songs that do not meet the specified metadata criteria. The search process continues until we collect the required number of songs that satisfy both similarity and metadata conditions.


A self-querying retriever is a retriever that possesses the unique capability of querying itself. True to its name, this retriever can take any natural language query and use a query-constructing Language Model (LLM) chain to generate a structured query. It then applies this structured query to its underlying VectorStore. Consequently, the retriever can not only perform semantic similarity comparisons between the user's input query and the contents of stored documents but also extract filters from the user's query based on the metadata of these stored documents. Additionally, the retriever can execute these extracted filters to further refine the search results.



In this notebook we explain how to approach the retriever pattern of semantice search with metadata filtering leveraging self quering with ChromaDB. 

Chroma is the open-source embedding database. Chroma makes it easy to build LLM apps by making knowledge, facts, and skills pluggable for LLMs.

Chroma gives you the tools to:

* store embeddings and their metadata
* embed documents and queries
* search embeddings

Chroma prioritizes:

* simplicity and developer productivity
* analysis on top of search
* it also happens to be very quick

## Usecase
#### Dataset
In this example, you will use several years of Amazon's Letter to Shareholders as a text corpus to perform Q&A on.

#### Persona
Let's assume a persona of a customer trying to ask various questions  based on different criterias.

## Implementation
In order to follow the RAG approach this notebook is using the LangChain framework where it has integrations with different services and tools that allow efficient building of patterns such as RAG. We will be using the following tools:

- **LLM (Large Language Model)**: Anthropic Claude V1 available through Amazon Bedrock

  This model will be used to understand the document chunks and provide an answer in human friendly manner.
- **Embeddings Model**: Amazon Titan Embeddings available through Amazon Bedrock

  This model will be used to generate a numerical representation of the textual documents
- **Document Loader**: PDF Loader available through LangChain

  This is the loader that can load the documents from a source, for the sake of this notebook we are loading the sample files from a local path. This could easily be replaced with a loader to load documents from enterprise internal systems.

- **Vector Store**: ChromDB available through LangChain

  In this notebook we are using ChromaDB vector-store to store both the embeddings and the documents. 
- **Index**: VectorIndex

  The index helps to compare the input embedding and the document embeddings to find relevant document
- **Wrapper**: wraps index, vector store, embeddings model and the LLM to abstract away the logic from the user.

### Setup
To run this notebook you would need to install 2 more dependencies, Lark and ChromaDB

Then begin with instantiating the LLM and the Embeddings model. Here we are using Amazon Titan to demonstrate the use case.

Note: It is possible to choose other models available with Bedrock. You can replace the `model_id` as follows to change the model.

`llm = Bedrock(model_id="amazon.titan-tg1-large")`

Available models under Bedrock have the following IDs:
- `amazon.titan-tg1-large`
- `ai21.j2-grande-instruct`
- `ai21.j2-jumbo-instruct`
- `anthropic.claude-instant-v1`
- `anthropic.claude-v1`



## Install dependencies

This dependencies script in the root of this repo provides a set of boto3 and cli resources to allow for Bedrock use while in preview.

In [ ]:
!cd .. && ./download-dependencies.sh

In [ ]:
import glob
import subprocess

botocore_whl_filename = glob.glob("../dependencies/botocore-*-py3-none-any.whl")[0]
boto3_whl_filename = glob.glob("../dependencies/boto3-*-py3-none-any.whl")[0]

subprocess.Popen(['pip', 'install', botocore_whl_filename, boto3_whl_filename, '--force-reinstall'], bufsize=1, universal_newlines=True)

This notebook demonstrates invoking Bedrock models directly using the AWS SDK, but for later notebooks in the workshop you'll also need to install [LangChain](https://github.com/hwchase17/langchain):

In [ ]:
%pip install langchain --quiet

### Install ChromaDB and Lark packages
The self-query retriever requires you to have  [chromadb](https://docs.trychroma.com/) package and lark. [Lark](https://lark-parser.readthedocs.io/en/stable/) is a modern general-purpose parsing library for Python. With Lark, you can parse any context-free grammar, efficiently, with very little code.

In [ ]:
%pip install chromadb --force-reinstall

In [ ]:
%pip install lark

In [ ]:
%pip install pydantic==1.10.12 --force-reinstall 

Next we need to install Langchain

In [ ]:
%pip install langchain --force-install

In [ ]:
%pip install pypdf==3.14.0

## Prepare the data

In this example, you will use several years of Amazon's Letter to Shareholders as a text corpus to perform Q&A on.

First you will download these files from the internet.

In [ ]:
!mkdir -p ./data

from urllib.request import urlretrieve
urls = [
    'https://s2.q4cdn.com/299287126/files/doc_financials/2023/ar/2022-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2022/ar/2021-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2021/ar/Amazon-2020-Shareholder-Letter-and-1997-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2020/ar/2019-Shareholder-Letter.pdf'
]

filenames = [
    'AMZN-2022-Shareholder-Letter.pdf',
    'AMZN-2021-Shareholder-Letter.pdf',
    'AMZN-2020-Shareholder-Letter.pdf',
    'AMZN-2019-Shareholder-Letter.pdf'
]

metadata = [
    dict(year=2022, source=filenames[0]),
    dict(year=2021, source=filenames[1]),
    dict(year=2020, source=filenames[2]),
    dict(year=2019, source=filenames[3])]

data_root = "./data/"

for idx, url in enumerate(urls):
    file_path = data_root + filenames[idx]
    urlretrieve(url, file_path)

As part of Amazon's culture, the CEO always includes a copy of the 1997 Letter to Shareholders with every new release. This will cause repetition, take longer to generate embeddings, and may skew your results. In the next section you will take the downloaded data, trim the 1997 letter (last 3 pages) and overwrite them as processed files.

In [ ]:
import glob
from pypdf import PdfReader, PdfWriter

local_pdfs = glob.glob(data_root + '*.pdf')

for local_pdf in local_pdfs:
    pdf_reader = PdfReader(local_pdf)
    pdf_writer = PdfWriter()
    for pagenum in range(len(pdf_reader.pages)-3):
        page = pdf_reader.pages[pagenum]
        pdf_writer.add_page(page)

    with open(local_pdf, 'wb') as new_file:
        new_file.seek(0)
        pdf_writer.write(new_file)
        new_file.truncate()


Now that you have clean PDFs to work with, you will enrich your documents with metadata, then use a process called "chunking" to break up a larger document into small pieces. These small pieces will allow you to generate embeddings without surpassing the input limit of the embedding model.

In this example you will break the document into 1000 character chunks, with a 100 character overlap. This will allow your embeddings to maintain some of its context.

In [ ]:
import numpy as np
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

documents = []

for idx, file in enumerate(filenames):
    loader = PyPDFLoader(data_root + file)
    document = loader.load()
    for document_fragment in document:
        document_fragment.metadata = metadata[idx]
        
    print(f'{len(document)} {document}\n')
    documents += document

# - in our testing Character split works better with this PDF data set
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 500,
    chunk_overlap  = 100,
)

docs = text_splitter.split_documents(documents)

### Import all the necessary libraries and access bedrock

We will import all the necessary libraries and access bedrock

Interaction with the Bedrock API is done via boto3 SDK. To create a the Bedrock client, we are providing an utility method that supports different options for passing credentials to boto3. 
If you are running these notebooks from your own computer, make sure you have [installed the AWS CLI](https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html) before proceeding.


#### Use default credential chain

If you are running this notebook from a Sagemaker Studio notebook and your Sagemaker Studio role has permissions to access Bedrock you can just run the cells below as-is. This is also the case if you are running these notebooks from a computer whose default credentials have access to Bedrock

#### Use a different role

In case you or your company has setup a specific role to access Bedrock, you can specify such role by uncommenting the line `#os.environ['BEDROCK_ASSUME_ROLE'] = '<YOUR_VALUES>'` in the cell below before executing it. Ensure that your current user or role have permissions to assume such role.

#### Use a specific profile

In case you are running this notebooks from your own computer and you have setup the AWS CLI with multiple profiles and the profile which has access to Bedrock is not the default one, you can uncomment the line `#os.environ['AWS_PROFILE'] = '<YOUR_VALUES>'` and specify the profile to use.

#### Note about `langchain`

The Bedrock classes provided by `langchain` create a default Bedrock boto3 client. We recommend to explicitly create the Bedrock client using the instructions below, and pass it to the class instantiation methods using `client=boto3_bedrock`

In [ ]:
#### Un comment the following lines to run from your local environment outside of the AWS account with Bedrock access

#import os
#os.environ['BEDROCK_ASSUME_ROLE'] = '<YOUR_VALUES>'
#os.environ['AWS_PROFILE'] = 'bedrock-user'

In [ ]:
import boto3
import json
import os
import sys

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'
boto3_bedrock = bedrock.get_bedrock_client(os.environ.get('BEDROCK_ASSUME_ROLE', None))

In [ ]:
boto3_bedrock.list_foundation_models()

#### Creating a Chroma Vectorstore

First we'll want to create a Chroma VectorStore and seed it with some data. 

In this example, you will be using the Amazon Titan Embeddings Model from Amazon Bedrock to generate the embeddings for our Chroma vector database.

The `TokenCounterHandler` callback function is a function you can utilize in your LLM objects and chains to generate reports on token count. It is supplied here as a utility class that will output the token counts at the end of your result chain, or can be attached to a LLM object and invoked manually.

In [ ]:
%pip install tiktoken==0.4.0 --force-reinstall

In [ ]:
from utils.TokenCounterHandler import TokenCounterHandler

token_counter = TokenCounterHandler()

In [ ]:

from IPython.display import Markdown, display
from langchain.llms.bedrock import Bedrock 
from langchain.embeddings.bedrock import BedrockEmbeddings

embeddings = BedrockEmbeddings(model_id="amazon.titan-e1t-medium")



Next you will import the Document and ChromaDB modules from Langchain. Using these modules will allow you to quickly generate embeddings through Amazon Bedrock and store them locally in your ChromaDB vector store.

In [ ]:
from langchain.schema import Document
from langchain.vectorstores import Chroma

In this step you will process documents and prepare them to be converted to vectors for the vector store.

Here you will use the from_documents function in the Langchain Chroma provider to build a vector database from your document embeddings.

In [ ]:
db = Chroma.from_documents(docs, embeddings)

## Similarity Searching

Here you will set your search query, and look for documents that match.

In [ ]:
query = "How has AWS evolved?"

### Basic Similarity Search

The results that come back from the `similarity_search_with_score` API are sorted by score from lowest to highest. The score value is represented by the [L-squared (or L2)](https://en.wikipedia.org/wiki/Lp_space) distance of each result. Lower scores are better, repesenting a shorter distance between vectors.

In [ ]:
results_with_scores = db.similarity_search_with_score(query)
for doc, score in results_with_scores:
    print(f"Content: {doc.page_content}\nMetadata: {doc.metadata}\nScore: {score}\n\n")

### Similarity Search with Metadata Filtering
Additionally, you can provide metadata to your query to filter the scope of your results. The `filter` parameter for search queries is a dictionary of metadata key/value pairs that will be matched to results to include/exclude them from your query.

In [ ]:
filter = dict(year=2022)

In the next section, you will notice that your query has returned less results than the basic search, because of your filter criteria on the resultset.

In [ ]:
results_with_scores = db.similarity_search_with_score(query, filter=filter)
for doc, score in results_with_scores:
    print(f"Content: {doc.page_content}\nMetadata: {doc.metadata}, Score: {score}\n\n")

### Top-K Matching

Top-K Matching is a filtering technique that involves a 2 stage approach.

1. Perform a similarity search, returning the top K matches.
2. Apply your metadata filter on the smaller resultset.

Note: A caveat for Top-K matching is that if the value for K is too small, there is a chance that after filtering there will be no results to return.

Using Top-K matching requires 2 values:
- `k`, the max number of results to return at the end of our query
- `fetch_k`, the max number of results to return from the similarity search before applying filters


In [ ]:
results = db.similarity_search(query, filter=filter, k=2, fetch_k=4)
for doc in results:
    print(f"Content: {doc.page_content}\nMetadata: {doc.metadata}\n\n")

### Maximal Marginal Relevance

Another measurement of results is Maximal Marginal Relevance (MMR). The focus of MMR is to minimize the redundancy of your search results while still maintaining relevance by re-ranking the results to provide both similarity and diversity.

In the next section you will use the `max_marginal_relevance_search` API to run the same query as in the Metadata Filtering section, but with reranked results.

In [ ]:
results = db.max_marginal_relevance_search(query, filter=filter)
for doc in results:
    print(f"Content: {doc.page_content}\nMetadata: {doc.metadata}\n\n")

## Q&A with Anthropic Claude and Retrieved Vectors

Now that you are able to query from the vector store, you're ready to feed context into your LLM.

Using the LangChain wrapper for Bedrock, creating an object for the LLM can be done in a single line of code where you specify the model_id of the desired LLM (Claude V2 in this case), and any model level arguments.

In [ ]:
from langchain.llms.bedrock import Bedrock

model_kwargs_titan = { 
        "maxTokenCount": 512,
        "stopSequences": [],
        "temperature":0.0,  
        "topP":0.5
}

# Amazon Titan Model
llm = Bedrock(
    model_id="amazon.titan-tg1-large", 
    client=boto3_bedrock, 
    #model_kwargs=model_kwargs_titan,
    callbacks=[token_counter]
)

Since you have a model object set up, you can use it to get a baseline of what the LLM will produce without any provided context.

Something you will notice is with the prompt "How has AWS evolved?", the answer isn't bad, but its not exactly what you'd look for from the lens of an executive. You'd want to hear about how they approached things that led to evolution, whereas the baseline results are just facts that indicate change. Later in the notebook, you will provide context to get a more tailored answer.

In [ ]:
print(llm.predict("How has AWS evolved?"))

token_counter.report()

With your LLM ready to go, you'll create a prompt template to utilize context to answer a given question. Prompt formats will be different by model, so if you change your model you will also likely need to adjust your prompt.

In [ ]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt_template = """Human: Here is a set of context, contained in <context> tags:

<context>
{context}
</context>

Use the context to provide an answer to the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{question}

Assistant:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

With the LLM endpoint object created, you are ready to create your first chain!

This chain is a simple example using LangChain's RetrievalQA chain, which will:
- take a query as input
- generate query embeddings
- query the vector database for relevant document chunks based on the query embedding
- inject the context and original query into the prompt template
- invoke the LLM with the completed prompt
- return the LLM result

The [`stuff` chain type](https://python.langchain.com/docs/modules/chains/document/stuff) simply takes the context documents and inserts them into the prompt.

By setting `return_source_documents` to `True`, the LLM responses will also contain the document chunks from the vector database, to illustrate where the context came from.

In [ ]:
#token_counter.clear_report()

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 5, "filter": filter},
        callbacks=[token_counter]
    ),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT},
    callbacks=[token_counter]
)

Now that your chain is set up, you can supply queries to it and generate responses based on your source documents.

You'll note that the LLM response references the context documents provided, using them to formulate a response calling out things that were mentioned specifically by Amazon's CEO.

In [ ]:
query = "How has AWS evolved?"
result = qa({"query": query})

print(f'Query: {result["query"]}\n')
print(f'Result: {result["result"]}\n')
print(f'Context Documents: ')
for srcdoc in result["source_documents"]:
      print(f'{srcdoc}\n')

In [ ]:
query = "Why is Amazon successful?"
result = qa({"query": query})

print(f'Query: {result["query"]}\n')
print(f'Result: {result["result"]}\n')
print(f'Context Documents: ')
for srcdoc in result["source_documents"]:
      print(f'{srcdoc}\n')

In [ ]:
query = "What business challenges has Amazon experienced?"
result = qa({"query": query})

print(f'Query: {result["query"]}\n')
print(f'Result: {result["result"]}\n')
print(f'Context Documents: ')
for srcdoc in result["source_documents"]:
      print(f'{srcdoc}\n')

In [ ]:
query = "How was Amazon impacted by COVID-19?"
result = qa({"query": query})

print(f'Query: {result["query"]}\n')
print(f'Result: {result["result"]}\n')
print(f'Context Documents: ')
for srcdoc in result["source_documents"]:
      print(f'{srcdoc}\n')